In [1]:
from flask import Flask, request, jsonify
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#4-01-26T20:44:50.764490Z","iopub.execute_input":"2024-01-26T20:44:50.764987Z","iopub.status.idle":"2024-01-26T20:45:02.374046Z","shell.execute_reply.started":"2024-01-26T20:44:50.764954Z","shell.execute_reply":"2024-01-26T20:45:02.372894Z"}}


# %% [code] {"execution":{"iopub.status.busy":"2024-01-26T20:45:02.375499Z","iopub.execute_input":"2024-01-26T20:45:02.375819Z","iopub.status.idle":"2024-01-26T20:45:02.382170Z","shell.execute_reply.started":"2024-01-26T20:45:02.375789Z","shell.execute_reply":"2024-01-26T20:45:02.381146Z"}}
import gc
import re
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch
import chess

In [2]:
!python -m pip install python-chess

In [3]:
class module(nn.Module):

    def __init__(self, hidden_size):
        super(module, self).__init__()
        self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_size)
        self.bn2 = nn.BatchNorm2d(hidden_size)
        self.activation1 = nn.SELU()
        self.activation2 = nn.SELU()

    def forward(self, x):
        x_input = torch.clone(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += x_input
        x = self.activation2(x)
        return x
class ChessNet(nn.Module):

    def __init__(self, hidden_layers=4, hidden_size=200):
        super(ChessNet, self).__init__()
        self.hidden_layers = hidden_layers
        self.input_layer = nn.Conv2d(6, hidden_size, 3, stride=1, padding=1)
        self.module_list = nn.ModuleList([module(hidden_size) for i in range(hidden_layers)])
        self.output_layer = nn.Conv2d(hidden_size, 2, 3, stride=1, padding=1)

    def forward(self, x):

        x = self.input_layer(x)
        x = F.relu(x)

        for i in range(self.hidden_layers):
            x = self.module_list[i](x)

        x = self.output_layer(x)

        return x


In [4]:
letter_2_num = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
num_2_letter = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}

In [5]:
#making chess board layers
def create_rep_layer(board, type):

    s = str(board)
    s = re.sub(f'[^{type}{type.upper()} \n]', '.', s)
    s = re.sub(f'{type}', '-1', s)
    s = re.sub(f'{type.upper()}', '1', s)
    s = re.sub(f'\.', '0', s)

    board_mat = []
    for row in s.split('\n'):
        row = row.split(' ')
        row = [int(x) for x in row]
        board_mat.append(row)

    return np.array(board_mat)

In [6]:
#single move ki representation
def move_2_rep(move, board):
    board.push_san(move).uci()
    move = str(board.pop())

    from_output_layer = np.zeros((8,8))
    from_row = 8 - int(move[1])
    from_column = letter_2_num[move[0]]
    from_output_layer[from_row, from_column] = 1

    to_output_layer = np.zeros((8,8))
    to_row = 8 - int(move[3])
    tow_column = letter_2_num[move[2]]
    to_output_layer[to_row, tow_column] = 1

    return np.stack([from_output_layer, to_output_layer])

In [7]:
def board_2_rep(board):
    pieces = ['p', 'r', 'n', 'b', 'q', 'k']
    layers = []
    for piece in pieces:
        layers.append(create_rep_layer(board, piece))
    board_rep = np.stack(layers)
    return board_rep

In [8]:
def return_mat(board):
    notation_mapping = {   #standard to sidhu notation
        'r': 'b-rook',
        'n': 'b-knight',
        'b': 'b-bishop',
        'q': 'b-queen',
        'k': 'b-king',
        'p': 'b-pawn',
        'R': 'w-rook',
        'N': 'w-knight',
        'B': 'w-bishop',
        'Q': 'w-queen',
        'K': 'w-king',
        'P': 'w-pawn',
        '.': 'null'
    }
    board_str = str(board)
    # Replace the standard notation with custom notation
    new_board_chars = []
    for char in board_str:
        if char in notation_mapping:
            new_board_chars.append(str(notation_mapping[char]))
        else:
            new_board_chars.append(char)

    # Join the modified characters to form the new board string
    new_board_str = ' '.join(new_board_chars)

    # Print the modified board string
    return new_board_str

def input_board(board):
    # Convert the board string to a 2D list
    # board = [row.split() for row in board_str.split('\n')]

    reverse_notation_mapping = {
        'b-rook': 'r',
        'b-knight': 'n',
        'b-bishop': 'b',
        'b-queen': 'q',
        'b-king': 'k',
        'b-pawn': 'p',
        'w-rook': 'R',
        'w-knight': 'N',
        'w-bishop': 'B',
        'w-queen': 'Q',
        'w-king': 'K',
        'w-pawn': 'P',
        'null': '.'
    }

    # Convert custom notation to standard notation
    for i in range(8):
        for j in range(8):
            board[i][j] = reverse_notation_mapping[board[i][j]]

    # Convert the 2D list to a FEN string
    fen = ''
    for row in board:  # 8th row to 1st row
        empty_count = 0
        for square in row:  # 'a' to 'h'
            if square == '.':  # Empty square
                empty_count += 1
            else:  # Square is occupied by a piece
                if empty_count > 0:
                    fen += str(empty_count)
                    empty_count = 0
                fen += square
        if empty_count > 0:
            fen += str(empty_count)
        fen += '/'
    fen = fen[:-1]  # Remove the trailing '/'

    # Create a chess.Board object
    board = chess.Board(fen)
    return board


In [9]:
def checkmate_single(board):
    board = board.copy()
    legal_moves = list(board.legal_moves)
    for move in legal_moves:
        board.push_uci(str(move))
        if board.is_checkmate():
            move = board.pop()
            return move
        _ = board.pop()
    return None

# %% [code] {"execution":{"iopub.status.busy":"2024-01-26T20:45:02.592224Z","iopub.status.idle":"2024-01-26T20:45:02.592727Z","shell.execute_reply.started":"2024-01-26T20:45:02.592488Z","shell.execute_reply":"2024-01-26T20:45:02.592511Z"}}
def distribution_over_moves(vals):
    probs = np.array(vals)
    probs = np.exp(probs)
    probs = probs / probs.sum()
    probs = probs ** 3
    probs = probs / probs.sum()
    return probs

# %% [code] {"execution":{"iopub.status.busy":"2024-01-26T20:45:02.594041Z","iopub.status.idle":"2024-01-26T20:45:02.594945Z","shell.execute_reply.started":"2024-01-26T20:45:02.594715Z","shell.execute_reply":"2024-01-26T20:45:02.594739Z"}}
def predict(x):
    # Assuming model1.pth is in a different directory, for example, /content/drive/MyDrive/models/
    model_path = 'C://Users//Shreyash Verma//Desktop//CHESS_GAME//react-sharpening//chess-game//model1.pth'  # Update this with your actual path
    model = ChessNet(hidden_layers=4, hidden_size=200)
    model.load_state_dict(torch.load(model_path)) # Load from the updated path
    model = model.cuda()
    x = x.cuda()
    model.eval()
    with torch.no_grad():
        outputs = model(x)
        return outputs.cpu().numpy()


In [10]:
def choose_move(board,color):
    board = input_board(board)
    print(board)

    legal_moves = list(board.legal_moves)

    move = checkmate_single(board)   #checking if single move me possible hai check mate

    if move is not None:
        chosen_move_uci = move.uci()  # Convert to UCI format
        board.push_uci(chosen_move_uci)  # Now this should work
        b_mat = return_mat(board)   #if yes..kardo
        return b_mat

    x = torch.Tensor(board_2_rep(board)).float().to('cuda')
    if color == 'BLACK':
        x *= -1
    x = x.unsqueeze(0)
    move = predict(x)
    # print(move)
    vals = []
    froms = [str(legal_move)[:2] for legal_move in legal_moves]
    froms = list(set(froms))
    for from_ in froms:
        # print(move[0,:,:][0][0])
        val = move[0,:,:][0][8-int(from_[1]), letter_2_num[from_[0]]]
        # print(from_)
        vals.append(val)

    probs = distribution_over_moves(vals)

    chosen_from = str(np.random.choice(froms, size=1, p=probs)[0])[:2]

    vals = []
    for legal_move in legal_moves:
        from_ = str(legal_move)[:2]
        if from_ == chosen_from:
            to = str(legal_move)[2:]
            # print(move[0,:,:][0])
            # print(move[0,:,:][1])
            val = move[0,:,:][1][8 - int(to[1]), letter_2_num[to[0]]]
            vals.append(val)
        else:
            vals.append(0)
    chosen_move = legal_moves[np.argmax(vals)]
    # Create a new chess board
    # board = chess.Board()
    # Assuming chosen_move is your Move object
    chosen_move_uci = chosen_move.uci()  # Convert to UCI format
    board.push_uci(chosen_move_uci)  # Now this should work

# Push a move to the board
    # board.push_uci(chosen_move)
    b_mat = return_mat(board)
    print(b_mat)
    return b_mat

In [11]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [12]:
import chess
cboard = [
            ["b-rook", "b-knight", "b-bishop", "b-queen", "b-king", "b-bishop", "b-knight", "b-rook"],
            ["b-pawn", "b-pawn", "null", "b-pawn", "b-pawn", "b-pawn", "b-pawn", "b-pawn"],
            ["null", "null", "b-pawn", "null", "null", "null", "null", "null"],
            ["null", "null", "null", "null", "null", "null", "null", "null"],
            ["null", "null", "null", "null", "null", "null", "null", "null"],
            ["null", "null", "null", "w-pawn", "null", "null", "null", "null"],
            ["w-pawn", "w-pawn", "w-pawn", "w-pawn", "w-pawn", "w-pawn", "w-pawn", "w-pawn"],
            ["w-rook", "w-knight", "w-bishop", "w-queen", "w-king", "w-bishop", "w-knight", "w-rook"]
        ]
legal = choose_move(cboard,'WHITE')
cboard = legal

r n b q k b n r
p p . p p p p p
. . p . . . . .
. . . . . . . .
. . . . . . . .
. . . P . . . .
P P P P P P P P
R N B Q K B N R


C:\Users\Shreyash Verma\AppData\Local\Temp\ipykernel_5696\233013209.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)) # Load

b-rook   b-knight   b-bishop   b-queen   b-king   b-bishop   b-knight   b-rook 
 b-pawn   b-pawn   null   b-pawn   b-pawn   b-pawn   b-pawn   b-pawn 
 null   null   b-pawn   null   null   null   null   null 
 null   null   null   null   null   null   null   null 
 null   null   null   null   w-pawn   null   null   null 
 null   null   null   w-pawn   null   null   null   null 
 w-pawn   w-pawn   w-pawn   w-pawn   null   w-pawn   w-pawn   w-pawn 
 w-rook   w-knight   w-bishop   w-queen   w-king   w-bishop   w-knight   w-rook


In [13]:
# !python -m pip install Flask-Cors

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "http://localhost:5173"}})

# Load the pre-trained model
# model = ChessNet(hidden_layers=4, hidden_size=200)
# model.load_state_dict(torch.load('model1.pth'))
# model.eval()


@app.route('/play', methods=['POST'])
def play():
           
            chessboard = request.json['chessboard']
            print(type(chessboard))
            print(chessboard)
            new_board = choose_move(chessboard,'WHITE')
            # new_board=chessboard
            # print(type(new_board))
            return jsonify(new_board)

if __name__ == '__main__':
    app.run(port=5000)

# model = ChessNet(hidden_layers=4, hidden_size=200)
# model.load_state_dict(torch.load('model1.pth'))
# model.eval()

 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Nov/2024 10:29:31] "OPTIONS /play HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2024 10:29:31] "OPTIONS /play HTTP/1.1" 200 -


<class 'list'>
[['b-rook', 'b-knight', 'b-bishop', 'b-queen', 'b-king', 'b-bishop', 'b-knight', 'b-rook'], ['b-pawn', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['w-pawn', 'w-pawn', 'w-pawn', 'w-pawn', 'w-pawn', 'w-pawn', 'w-pawn', 'w-pawn'], ['w-rook', 'w-knight', 'w-bishop', 'w-queen', 'w-king', 'w-bishop', 'w-knight', 'w-rook']]
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
<class 'list'>
[['b-rook', 'b-knight', 'b-bishop', 'b-queen', 'b-king', 'b-bishop', 'b-knight', 'b-rook'], ['b-pawn', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 

C:\Users\Shreyash Verma\AppData\Local\Temp\ipykernel_5696\233013209.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)) # Load

...

<class 'list'>
[['b-rook', 'b-knight', 'b-bishop', 'b-queen', 'b-king', 'b-bishop', 'b-knight', 'b-rook'], ['b-pawn', 'b-pawn', 'b-pawn', 'null', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn'], ['null', 'null', 'null', 'b-pawn', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'w-pawn', 'w-pawn', 'null', 'null', 'null'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['w-pawn', 'w-pawn', 'w-pawn', 'null', 'null', 'w-pawn', 'w-pawn', 'w-pawn'], ['w-rook', 'w-knight', 'w-bishop', 'w-queen', 'w-king', 'w-bishop', 'w-knight', 'w-rook']]
r n b q k b n r
p p p . p p p p
. . . p . . . .
. . . . . . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
b-rook   b-knight   b-bishop   b-queen   b-king   b-bishop   b-knight   b-rook 
 b-pawn   b-pawn   b-pawn   null   b-pawn   b-pawn   b-pawn   b-pawn 
 null   null   null   b-pawn   null   null   null   null 
 null   null   null   null   null   nu

127.0.0.1 - - [09/Nov/2024 10:53:44] "OPTIONS /play HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2024 10:53:45] "POST /play HTTP/1.1" 200 -


<class 'list'>
[['b-rook', 'b-knight', 'b-bishop', 'null', 'b-king', 'b-bishop', 'b-knight', 'b-rook'], ['b-pawn', 'b-pawn', 'b-pawn', 'b-queen', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn'], ['null', 'null', 'null', 'b-pawn', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'w-pawn', 'w-pawn', 'null', 'null', 'null'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['w-pawn', 'w-pawn', 'w-pawn', 'null', 'null', 'w-pawn', 'w-pawn', 'w-pawn'], ['w-rook', 'w-knight', 'w-bishop', 'w-queen', 'w-king', 'w-bishop', 'w-knight', 'w-rook']]
r n b . k b n r
p p p q p p p p
. . . p . . . .
. . . . . . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
b-rook   b-knight   b-bishop   null   b-king   b-bishop   b-knight   b-rook 
 b-pawn   b-pawn   b-pawn   b-queen   b-pawn   b-pawn   b-pawn   b-pawn 
 null   null   null   b-pawn   null   null   null   null 
 null   null   null   null   null   nu

127.0.0.1 - - [09/Nov/2024 10:53:46] "POST /play HTTP/1.1" 200 -


<class 'list'>
[['b-rook', 'b-knight', 'b-bishop', 'null', 'b-king', 'b-bishop', 'b-knight', 'b-rook'], ['b-pawn', 'b-pawn', 'b-pawn', 'b-queen', 'b-pawn', 'b-pawn', 'b-pawn', 'b-pawn'], ['null', 'null', 'null', 'b-pawn', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null'], ['null', 'null', 'null', 'w-pawn', 'w-pawn', 'null', 'null', 'null'], ['null', 'null', 'w-knight', 'null', 'null', 'null', 'null', 'null'], ['w-pawn', 'w-pawn', 'w-pawn', 'null', 'null', 'w-pawn', 'w-pawn', 'w-pawn'], ['w-rook', 'null', 'w-bishop', 'w-queen', 'w-king', 'w-bishop', 'w-knight', 'w-rook']]
r n b . k b n r
p p p q p p p p
. . . p . . . .
. . . . . . . .
. . . P P . . .
. . N . . . . .
P P P . . P P P
R . B Q K B N R
b-rook   b-knight   b-bishop   null   b-king   b-bishop   b-knight   b-rook 
 b-pawn   b-pawn   b-pawn   b-queen   b-pawn   b-pawn   b-pawn   b-pawn 
 null   null   null   b-pawn   null   null   null   null 
 null   null   null   null   null   nu